# <font color='red'> 필요한 패키지 설정

In [0]:
from google.colab import drive
from datetime import datetime

import os
import pandas as pd

# <font color='red'> 데이터 로딩

In [0]:
# 구글 드라이브 접근 권한 획득
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
# 구글 드라이브 접근 확인하기
!ls -lt "/content/gdrive/My Drive/ML_basic/ML_basic_dataset"

total 4576455
-rw------- 1 root root      41253 Jul 16 02:15 LCDataDictionary_modification_3.xlsx
-rw------- 1 root root 1441750553 Jul 15 02:01 loan_seg_df_3.pkl
-rw------- 1 root root  836188793 Jul 14 14:11 loan_seg_df_2.pkl
-rw------- 1 root root 1218773813 Jul 14 09:27 loan_seg_df_1.pkl
-rw------- 1 root root      40581 Jul 14 07:03 LCDataDictionary_modification_2.xlsx
-rw------- 1 root root      39139 Jul 13 16:27 LCDataDictionary_modification_1.xlsx
-rw------- 1 root root      33741 Jul 13 09:46 LCDataDictionary_modification.xlsx
-rw------- 1 root root      23594 Jul 13 09:01 LCDataDictionary.xlsx
-rw------- 1 root root 1189395649 Jun 30 12:21 loan.csv


In [0]:
# 피클파일 읽기
folder_path = r'/content/gdrive/My Drive/ML_basic/ML_basic_dataset'
pkl_fname = 'loan_seg_df_3.pkl'
pkl_fpath = folder_path + os.sep + pkl_fname
print('파일경로: ', pkl_fpath)
loan_seg_df = pd.read_pickle(pkl_fpath)
print('파일형태: ', loan_seg_df.shape)
loan_seg_df.head(2)

파일경로:  /content/gdrive/My Drive/ML_basic/ML_basic_dataset/loan_seg_df_3.pkl
파일형태:  (1962274, 95)


id member_id  loan_amnt  ...  loan_status_curr_fg  issue_yymmdd issue_yymm
186                    4500  ...                False    2018-12-01     201812
269                   20000  ...                False    2018-12-01     201812

[2 rows x 95 columns]

In [0]:
# 인덱스를 새롭게 생성합니다.
loan_seg_df.reset_index(drop=True, inplace=True)
loan_seg_df.head(2)

id member_id  loan_amnt  ...  loan_status_curr_fg  issue_yymmdd issue_yymm
0                    4500  ...                False    2018-12-01     201812
1                   20000  ...                False    2018-12-01     201812

[2 rows x 95 columns]

In [0]:
loan_half_m_df = loan_seg_df.head(500000).copy()
loan_half_m_df.shape

(500000, 95)

# <font color= 'red'> 새로운 변수 생성

## 행별로 함수를 반복 적용하는 방법

* 함수란?
  * ml_basic03_programming_3_reference.ipynb 참고
* DataFrame의 <font color = 'red'> **행**</font>을 순회하면서 특정 함수를 <font color='red'>**반복 적용**</font>하는 방법
  * ~~for loop using df.iloc[]~~
```python
  for idx_seq in range(df.shape[0]):
      row['new_col'] = create_new_col(df.iloc[idx_seq]['col1'], df.iloc[idx_seq]['col2'])
```
  * ~~for loop using df.iterrows()~~
```python
  for idx, row in df.iterrows():
      row['new_col'] = create_new_col(row['col1'], row['col2'])
```
  * apply method
```python
  df['new_col'] = df.apply(lambda row: create_new_col(row['col1'], row['col2']), axis=1)
```
  * vectorization
```python
  df['new_col'] = create_new_col(row['col1'], row['col2'])
```



### apply 함수

* DataFrame 사용 시, apply 함수와 map 함수
  * map 함수: 1개의 Series에만 적용 가능
  * apply 함수: 1개 이상의 Series에 적용 가능
    * 1개의 Series에 apply 함수를 적용하면? > map과 동일한 결과 산출됨
    * 따라서, apply 함수만 사용하겠습니다.
    * [DataFrame.apply](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html?highlight=apply#pandas.DataFrame.apply)

* lambda 함수
  * 한줄로 표현하는 익명의 함수이다. > 한번만 사용가능하다.
  * map 또는 apply 함수 등과 함께 사용되는 경우가 많다. 이해가 안되면 외우자!  

#### 케이스1) 입력인자 1개, 출력값 1개 함수

In [0]:
# 함수 선언
def apply_func_one_one(open_acc):
  new_col = open_acc * 10
  return new_col

In [0]:
s1 = datetime.now()
loan_half_m_df['new_col'] = loan_half_m_df.apply(lambda each_row: apply_func_one_one(each_row['open_acc']), axis=1)
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','new_col']].head(2)

# loan_half_m_df.apply > 모든 컬럼을 참조함 > 시간 & 공간많이 사용됨
# 순간적으로 사용 공간(메모리)가 늘어나는 것을 확인할 수 있음
# apply함수에 DataFrame이 입력됨 > 따라서, apply함수의 axis인자(행?, 열?)를 설정해야 함

11.833842


open_acc  new_col
0      12.0    120.0
1      10.0    100.0

In [0]:
s1 = datetime.now()
loan_half_m_df['new_col'] = loan_half_m_df['open_acc'].apply(lambda each_value: apply_func_one_one(each_value))
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','new_col']].head(2)

# loan_half_m_df['open_acc'].apply > 필요한 컬럼만 참조함 > 시간 & 공간 절약됨
# apply함수에 Series가 입력됨 > 따라서, apply함수의 axis인자(행?, 열?)를 설정할 필요 없음

0.235312


open_acc  new_col
0      12.0    120.0
1      10.0    100.0

In [0]:
# 해당 예시는 단순한 계산만 처리하므로 함수를 선언할 필요 없음 
#   > 단순한 계산이 아닌 경우에는 함수 선언 필요.
#   > 이와 같은 단순한 연산에 대해서는, 이후 vectorization에서 더 다룰 예정
s1 = datetime.now()
loan_half_m_df['new_col'] = loan_half_m_df['open_acc'] * 10
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','new_col']].head(2)

0.041765


open_acc  new_col
0      12.0    120.0
1      10.0    100.0

#### 케이스2) 입력인자 2개, 출력값 1개 함수

In [0]:
# 함수 선언
def apply_func_two_one(open_acc, total_acc):
  new_col = open_acc + total_acc
  return new_col

In [0]:
s1 = datetime.now()
loan_half_m_df['new_col'] = loan_half_m_df.apply(lambda x: apply_func_two_one(x['open_acc'], x['total_acc']), axis=1)
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','total_acc', 'new_col']].head(2)

15.790596


open_acc  total_acc  new_col
0      12.0       25.0     37.0
1      10.0       16.0     26.0

In [0]:
s1 = datetime.now()
loan_half_m_df['new_col'] = loan_half_m_df[['open_acc','total_acc']].apply(lambda x: apply_func_two_one(x['open_acc'], x['total_acc']), axis=1)
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','total_acc', 'new_col']].head(2)

11.514098


open_acc  total_acc  new_col
0      12.0       25.0     37.0
1      10.0       16.0     26.0

#### 케이스3) 입력인자 2개, 출력값 2개 함수

##### return Series

In [0]:
def func_two_two(open_acc, total_acc):
  new_col1 = open_acc + total_acc
  new_col2 = open_acc - total_acc
  return pd.Series([new_col1, new_col2])

# 복수개의 출력값을 DataFrame에 넘겨주기 위해서는 Series형태로 변환해야 한다.
# 매번 return할때마다 Series를 생성하는 연산이 포함되므로 시간이 오래 걸린다.

In [0]:
s1 = datetime.now()
loan_half_m_df[['new_col1','new_col2']] = loan_half_m_df.apply(lambda x: func_two_two(x['open_acc'], x['total_acc']), axis=1)
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','total_acc', 'new_col1', 'new_col2']].head(2)

# loan_half_m_df.apply > apply함수에 모든 컬럼이 적용되므로 시간/공간이 많이 사용된다.
# return을 복수개하므로 Series 생성 연산이 많아서 시간이 오래 걸린다.

169.320504


open_acc  total_acc  new_col1  new_col2
0      12.0       25.0      37.0     -13.0
1      10.0       16.0      26.0      -6.0

In [0]:
s1 = datetime.now()
loan_half_m_df[['new_col1','new_col2']] = loan_half_m_df[['open_acc','total_acc']].apply(lambda x: func_two_two(x['open_acc'], x['total_acc']), axis=1)
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','total_acc', 'new_col1', 'new_col2']].head(2)

# return을 복수개하므로 Series 생성 연산이 많아서 시간이 오래 걸린다.

128.327949


open_acc  total_acc  new_col1  new_col2
0      12.0       25.0      37.0     -13.0
1      10.0       16.0      26.0      -6.0

##### return tuple

In [0]:
def func_two_two_tuple(open_acc, total_acc):
  new_col1 = open_acc + total_acc
  new_col2 = open_acc - total_acc
  return new_col1, new_col2

# return할때 tuple로 넘기므로 시간이 절약된다.

In [0]:
s1 = datetime.now()
loan_half_m_df[['new_col1','new_col2']] = loan_half_m_df[['open_acc','total_acc']].apply(lambda x: func_two_two_tuple(x['open_acc'], x['total_acc']), axis=1, result_type='expand')
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','total_acc', 'new_col1', 'new_col2']].head(2)

# 함수 결과가 tuple로 넘어오므로 apply함수의 result_type인자를 expand로 설정해야 한다.

67.773608


open_acc  total_acc  new_col1  new_col2
0      12.0       25.0      37.0     -13.0
1      10.0       16.0      26.0      -6.0

##### return row

In [0]:
def func_two_two_row(row):
  row['new_col1'] = row['open_acc'] + row['total_acc']
  row['new_col2'] = row['open_acc'] - row['total_acc']
  return row

# return할때 row를 넘기므로 함수결과를 받는 쪽에서 편하다.

In [0]:
tmp_df = loan_half_m_df.copy()
s1 = datetime.now()
tmp_df = tmp_df[['open_acc','total_acc']].apply(lambda x: func_two_two_row(x), axis=1)
s2 = datetime.now()
print((s2-s1).total_seconds())
tmp_df.head(2)

# apply 함수에 필요한 컬럼만 넘겼으므로 다른 컬럼은 사려진다.
# row 전체를 교체해야하므로 시간이 오래 걸린다.

1070.071515


open_acc  total_acc  new_col1  new_col2
0      12.0       25.0      37.0     -13.0
1      10.0       16.0      26.0      -6.0

In [0]:
s1 = datetime.now()
loan_half_m_df = loan_half_m_df.apply(lambda x: func_two_two_row(x), axis=1)
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','total_acc', 'new_col1', 'new_col2']].head(2)

# 함수가 row를 return할때는 apply 함수 사용시, 모든 컬럼을 넘겨야 한다.
# 게다가 row 전체를 교체해야하므로 시간이 오래 걸린다.
# 시간/메모리가 많이 사용된다. 하지만 코드가 간결하다.

123.807878


open_acc  total_acc  new_col1  new_col2
0      12.0       25.0      37.0     -13.0
1      10.0       16.0      26.0      -6.0

#### apply 함수 결론

* apply 함수의 인자로 필요한 컬럼만 입력한다.
  * 필요한 컬럼만 접근하므로 컬럼 데이터가 저장된 공간(메모리) 접근 빈도가 줄어든다. (time complexity)
  * 필요한 컬럼만 불러오므로 메모리 사용량이 줄어든다. (space complexity)
* 복수개의 컬럼을 return하는 함수 정의 시, tuple로 return하자  
  * apply 함수의 result_type='expand' 설정 필요
  * series로 return하면
    * 매번 series 객체를 생성해야하므로 시간이 오래 걸린다. (time complexity)
  * row로 return하면
    * 매번 row 전체(모든 컬럼)을 인자로 넘겨야 하므로 시간/공간이 많이 사용된다. (time/space complexity)
    * 매번 row 전체를 update해야 하므로 시간이 오래 걸린다. (time complexity)

### Vectorization

* Vectorization 참고자료
  * [A Beginner’s Guide to Optimizing Pandas Code for Speed](https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6)
  * [VECTORIZATION AND PARALLELIZATION IN PYTHON WITH NUMPY AND PANDAS](https://datascience.blog.wzb.eu/2018/02/02/vectorization-and-parallelization-in-python-with-numpy-and-pandas/)

![vectorization](https://docs.google.com/uc?export=download&id=1-RPF97erGS3Sbk09IQQe0aduICUW6dQT)

#### 케이스1) 입력인자 2개, 출력값 1개 함수

cola * colb
a + b
a[0] + b[0]
a[1] + b[1]

##### vector 함수 정의 후 적용

In [0]:
def vec_func_one_output(open_acc, total_acc):
  new_col = open_acc + total_acc
  return new_col

In [0]:
s1 = datetime.now()
loan_half_m_df['new_col'] = vec_func_one_output(loan_half_m_df['open_acc'], loan_half_m_df['total_acc'])
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','total_acc', 'new_col']].head(2)

0.014634


open_acc  total_acc  new_col
0      12.0       25.0     37.0
1      10.0       16.0     26.0

##### 암묵적 vector 함수 적용

In [0]:
s1 = datetime.now()
loan_half_m_df['new_col'] = loan_half_m_df['open_acc'] + loan_half_m_df['total_acc']
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','total_acc', 'new_col']].head(2)

0.01061


open_acc  total_acc  new_col
0      12.0       25.0     37.0
1      10.0       16.0     26.0

#### 케이스2) 입력인자2개, 출력갑 2개 함수

##### vector 함수 정의 후 적용

In [0]:
def vec_func_two_outputs(open_acc, total_acc):
  new_col1 = open_acc + total_acc
  new_col2 = open_acc - total_acc
  return new_col1, new_col2

In [0]:
s1 = datetime.now()
loan_half_m_df['new_col1'], loan_half_m_df['new_col2'] = vec_func_two_outputs(loan_half_m_df['open_acc'], loan_half_m_df['total_acc'])
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','total_acc', 'new_col1', 'new_col2']].head(2)

0.020461


open_acc  total_acc  new_col1  new_col2
0      12.0       25.0      37.0     -13.0
1      10.0       16.0      26.0      -6.0

##### 암묵적 vector 함수 적용

In [0]:
s1 = datetime.now()
loan_half_m_df['new_col1'] = loan_half_m_df['open_acc'] + loan_half_m_df['total_acc']
loan_half_m_df['new_col2'] = loan_half_m_df['open_acc'] - loan_half_m_df['total_acc']
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['open_acc','total_acc', 'new_col1', 'new_col2']].head(2)

0.019403


open_acc  total_acc  new_col1  new_col2
0      12.0       25.0      37.0     -13.0
1      10.0       16.0      26.0      -6.0

#### 케이스3) 문자열 처리 함수

##### vectorization이 불가능한 경우

In [0]:
def vec_func_string_manipulation(date_str, date_format):
  dtim_obj = datetime.strptime(date_str, date_format)
  dtim_yy = dtim_obj.year
  dtim_mm = dtim_obj.month
  return dtim_yy, dtim_mm

In [0]:
# 최초대출개설년월
s1 = datetime.now()
loan_half_m_df['earliest_cr_line_yy'], loan_half_m_df['earliest_cr_line_mm'] = vec_func_string_manipulation(loan_half_m_df['earliest_cr_line'], '%b-%Y')
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['earliest_cr_line','earliest_cr_line_yy', 'earliest_cr_line_mm']].head(2)

# datetime.strptime함수(연산)는 vectorization이 가능하지 않습니다. 
# 따라서, 함수 정의를 변경하여 vectorization이 가능하도록 해야 합니다.

TypeError: ignored

##### vectorization을 위한 함수 재정의

In [0]:
def vec_func_string_manipulation(date_str, date_format):
  dtim_obj = date_str.map(lambda x: datetime.strptime(x, date_format))
  dtim_yy = dtim_obj.map(lambda x: x.year)
  dtim_mm = dtim_obj.map(lambda x: x.month)
  return dtim_yy, dtim_mm

In [0]:
# 최초대출개설년월
s1 = datetime.now()
loan_half_m_df['earliest_cr_line_yy'], loan_half_m_df['earliest_cr_line_mm'] = vec_func_string_manipulation(loan_half_m_df['earliest_cr_line'], '%b-%Y')
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['earliest_cr_line','earliest_cr_line_yy', 'earliest_cr_line_mm']].head(2)

7.877779


earliest_cr_line  earliest_cr_line_yy  earliest_cr_line_mm
0         Dec-2003                 2003                   12
1         Apr-1995                 1995                    4

##### apply 함수 사용

In [0]:
# 다른 방법1
s1 = datetime.now()
loan_half_m_df['earliest_cr_line_dtim'] = loan_half_m_df['earliest_cr_line'].apply(lambda x: datetime.strptime(x, '%b-%Y'))
loan_half_m_df['earliest_cr_line_yy'] = loan_half_m_df['earliest_cr_line_dtim'].apply(lambda x: x.year)
loan_half_m_df['earliest_cr_line_mm'] = loan_half_m_df['earliest_cr_line_dtim'].apply(lambda x: x.month)
s2 = datetime.now()
print((s2-s1).total_seconds())
loan_half_m_df[['earliest_cr_line','earliest_cr_line_yy', 'earliest_cr_line_mm']].head(2)

8.209692


earliest_cr_line  earliest_cr_line_yy  earliest_cr_line_mm
0         Dec-2003                 2003                   12
1         Apr-1995                 1995                    4

In [0]:
# 다른 방법2
s1 = datetime.now()
loan_half_m_df['earliest_cr_line_dtim'] = loan_half_m_df['earliest_cr_line'].apply(lambda x: pd.to_datetime(x, format='%b-%Y'))
loan_half_m_df['earliest_cr_line_yy'] = loan_half_m_df['earliest_cr_line_dtim'].apply(lambda x: x.year)
loan_half_m_df['earliest_cr_line_mm'] = loan_half_m_df['earliest_cr_line_dtim'].apply(lambda x: x.month)
s2 = datetime.now()
loan_half_m_df[['earliest_cr_line','earliest_cr_line_yy', 'earliest_cr_line_mm']].head(2)

earliest_cr_line  earliest_cr_line_yy  earliest_cr_line_mm
0         Dec-2003                 2003                   12
1         Apr-1995                 1995                    4

## 행별로 함수를 반복 적용하는 방법 결론

* 초급: apply함수를 사용하세요.
* 중급: 암묵적 vector함수(연산)을 사용하세요.
* 고급: 명시적 vector함수를 정의하여 사용하세요.

# <font color = 'red'> 병합(merge)

* DataFrame 2개를 병합하는 함수
  * [DataFrame.merge()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html?highlight=merge#pandas.DataFrame.merge)
  * [Merge, join, and concatenate](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)
  * DataFrame.join()함수의 기능은 merge로 모두 구현 가능하므로 join은 다루지 않습니다.
* merge 유형
  
![merge_method](https://docs.google.com/uc?export=download&id=1h6zxloRWvgOAGmKUk73pwVX7zCh3kRs4)

## 인덱스를 활용한 merge

### 실습 데이터 생성

In [2]:
temp_df1 = pd.DataFrame(
    data={
        'col1': [1, 2, 3],
        'col2': ['a', 'b', 'c'],
    },
    index=[1, 2, 3]
)
temp_df1

col1 col2
1     1    a
2     2    b
3     3    c

In [3]:
temp_df2 = pd.DataFrame(
    data={
      'col3': [55, 33, 11],
      'col2': ['E', 'C', 'A'],
    },
    index=[5,3,1]
)
temp_df2

col3 col2
5    55    E
3    33    C
1    11    A

### merge type별 merge

In [4]:
# inner
merged_df = temp_df1.merge(temp_df2, left_index=True, right_index=True, how='inner', sort=True, indicator=True)
merged_df

col1 col2_x  col3 col2_y _merge
1     1      a    11      A   both
3     3      c    33      C   both

In [0]:
# outer
merged_df = temp_df1.merge(temp_df2, left_index=True, right_index=True, how='outer', sort=True, indicator=True)
merged_df

col1 col2_x  col3 col2_y      _merge
1   1.0      a  11.0      A        both
2   2.0      b   NaN    NaN   left_only
3   3.0      c  33.0      C        both
5   NaN    NaN  55.0      E  right_only

In [9]:
# left
merged_df = temp_df1.merge(temp_df2, left_index=True, right_index=True, how='left', sort=True, indicator=True)
merged_df

ValueError: ignored

In [0]:
# right
merged_df = temp_df1.merge(temp_df2, left_index=True, right_index=True, how='right', sort=True, indicator=True)
merged_df

col1 col2_x  col3 col2_y      _merge
1   1.0      a    11      A        both
3   3.0      c    33      C        both
5   NaN    NaN    55      E  right_only

### validate

In [5]:
# index 중복이 있는 경우
temp_df3 = pd.DataFrame(
    data={
      'col3': [55, 33, 33, 11],
      'col2': ['E', 'C', 'C', 'A'],
    },
    index=[5,3,3,1]
)
temp_df3

col3 col2
5    55    E
3    33    C
3    33    C
1    11    A

In [0]:
# one_to_one
merged_df = temp_df1.merge(temp_df3, left_index=True, right_index=True, how='inner', sort=True, indicator=True, validate='one_to_one')
merged_df

MergeError: ignored

In [6]:
# one_to_many
merged_df = temp_df1.merge(temp_df3, left_index=True, right_index=True, how='inner', sort=True, indicator=True, validate='one_to_many')
merged_df

col1 col2_x  col3 col2_y _merge
1     1      a    11      A   both
3     3      c    33      C   both
3     3      c    33      C   both

### suffixes

In [0]:
# 중복컬럼이 존재하는 경우: default suffix
merged_df = temp_df1.merge(temp_df2, left_index=True, right_index=True, how='inner', sort=True, indicator=True, validate='one_to_one', suffixes=('_x', '_y'))
merged_df

col1 col2_x  col3 col2_y _merge
1     1      a    11      A   both
3     3      c    33      C   both

In [0]:
# 중복컬럼이 존재하는 경우: custom suffix
merged_df = temp_df1.merge(temp_df2, left_index=True, right_index=True, how='inner', sort=True, indicator=True, validate='one_to_one', suffixes=('_1', '_2'))
merged_df

col1 col2_1  col3 col2_2 _merge
1     1      a    11      A   both
3     3      c    33      C   both

In [0]:
# 중복컬럼이 존재하는 경우: 원본 유지
merged_df = temp_df1.merge(temp_df2, left_index=True, right_index=True, how='inner', sort=True, indicator=True, validate='one_to_one', suffixes=('', '_merged'))
merged_df

col1 col2  col3 col2_merged _merge
1     1    a    11           A   both
3     3    c    33           C   both

In [0]:
# 중복컬럼을 허용하고 싶지 않는 경우
merged_df = temp_df1.merge(temp_df2, left_index=True, right_index=True, how='inner', sort=True, indicator=True, validate='one_to_one', suffixes=(False, False))
merged_df

ValueError: ignored

## 컬럼을 활용한 merge

### 실습 데이터 생성

In [7]:
temp_df1 = pd.DataFrame(
    data={
        'col1': [1, 2, 3],
        'col2': [0, 0, 1],
        'col3': ['a', 'b', 'c'],
    },
    index=[1, 2, 3]
)
temp_df1

col1  col2 col3
1     1     0    a
2     2     0    b
3     3     1    c

In [8]:
temp_df2 = pd.DataFrame(
    data={
      'col1': [5, 3, 1],
      'col2': [0, 1, 1],
      'col4': ['E', 'C', 'A'],
    },
    index=[5,3,1]
)
temp_df2

col1  col2 col4
5     5     0    E
3     3     1    C
1     1     1    A

### merge

In [0]:
# 1개 key 이용
merged_df = temp_df1.merge(temp_df2, left_on=['col1'], right_on=['col1'], how='inner', sort=True, indicator=True)
merged_df

col1  col2_x col3  col2_y col4 _merge
0     1       0    a       1    A   both
1     3       1    c       1    C   both

In [0]:
# 2개 key 이용
merged_df = temp_df1.merge(temp_df2, left_on=['col1','col2'], right_on=['col1', 'col2'], how='inner', sort=True, indicator=True)
merged_df

col1  col2 col3 col4 _merge
0     3     1    c    C   both

# <font color = 'red'> 실습 예제

In [10]:
applicantion_df = pd.DataFrame(
    data = {
        'customer_id': ['1111', '2222', '3333', '4444'],
        'apply_loan_amt': [2000, 2500, 3000, 3000],
    }
)
applicantion_df

customer_id  apply_loan_amt
0        1111            2000
1        2222            2500
2        3333            3000
3        4444            3000

In [11]:
income_df = pd.DataFrame(
    data = {
        'customer_id': ['1111', '1111', '4444'],
        'application_income': [60000, 50000, 80000],
        'certified_income': [55000, 45000, 75000]
    }
)
income_df

customer_id  application_income  certified_income
0        1111               60000             55000
1        1111               50000             45000
2        4444               80000             75000

* 신청고객 정보에 소득을 병합하세요.
  * 신청서소득정보(컬럼)와 증명소득정보(컬럼) 중, 작은 값 최종소득정보(컬럼)으로 생성하세요.
  * 동일 신청고객에 대해서 복수개의 최종소득정보 레코드가 존재하는 경우, 그 값이 큰 레코드를 사용하세요.
  * 병합 시, 소득정보가 병합되지 않는 경우, 0을 입력하세요.

## 솔루션1

In [0]:
# 최종소득 컬럼 생성
income_df['final_income'] = income_df[['application_income', 'certified_income']].apply(lambda x: min(x), axis=1)
income_df

customer_id  application_income  certified_income  final_income
0        1111               60000             55000         55000
1        1111               50000             45000         45000
2        4444               80000             75000         75000

In [12]:
def min_val(x, y):
  #gggggggg
  return min(x, y)

# 최종소득 컬럼 생성
income_df['final_income'] = income_df[['application_income', 'certified_income']].apply(lambda x: min_val(x['application_income'], x['certified_income']), axis=1)
income_df

customer_id  application_income  certified_income  final_income
0        1111               60000             55000         55000
1        1111               50000             45000         45000
2        4444               80000             75000         75000

In [0]:
# 중복 소득 존재 > 중복제거를 위해 정렬
income_df.sort_values(by=['customer_id', 'final_income'], axis=0, inplace=True, ascending=True)
income_df

customer_id  application_income  certified_income  final_income
1        1111               50000             45000         45000
0        1111               60000             55000         55000
2        4444               80000             75000         75000

In [0]:
# 중복 소득 제거
nodup_cols = ['customer_id']
nodup_income_df = income_df.drop_duplicates(subset=nodup_cols, inplace=False, keep='last')
nodup_income_df

customer_id  application_income  certified_income  final_income
0        1111               60000             55000         55000
2        4444               80000             75000         75000

In [0]:
# 병합
app_inc_df = applicantion_df.merge(nodup_income_df[['customer_id', 'final_income']], on=['customer_id'], how='left').fillna(0)
app_inc_df

customer_id  apply_loan_amt  final_income
0        1111            2000       55000.0
1        2222            2500           0.0
2        3333            3000           0.0
3        4444            3000       75000.0

## 솔루션2

In [0]:
# 최종소득 컬럼 생성
income_df['final_income'] = income_df[['application_income', 'certified_income']].apply(lambda x: min(x), axis=1)

# 소득 중복제거
nodup_income_df = income_df.groupby(['customer_id'], as_index=False).agg({'final_income': 'max'})

# 병합
app_inc_df = applicantion_df.merge(nodup_income_df[['customer_id', 'final_income']], on=['customer_id'], how='left').fillna(0)
app_inc_df

customer_id  apply_loan_amt  final_income
0        1111            2000       55000.0
1        2222            2500           0.0
2        3333            3000           0.0
3        4444            3000       75000.0